In [1]:
import numpy as np 
import pandas as pd 
import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

In [6]:
import tensorflow as tf
from tensorflow.keras.layers import Layer
from tensorflow.python.keras.utils.vis_utils import plot_model
import matplotlib.pyplot as plt

from IPython.display import SVG
from keras.utils.vis_utils import model_to_dot
import keras
from tensorflow.keras import datasets, layers, models, losses



In [11]:
(train_images, train_labels), (test_images, test_labels) = datasets.fashion_mnist.load_data()

In [12]:
X = train_images.astype(np.float32).reshape(-1, 28, 28, 1)  # 形状转换为 (60000, 28, 28, 1)
Y = train_labels

# 预处理测试数据
X_test = test_images.astype(np.float32).reshape(-1, 28, 28, 1)  # 形状转换为 (10000, 28, 28, 1)
Y_test = test_labels

print("X shape = ", X.shape)
print("Y shape = ", Y.shape)
print("X_test shape = ", X_test.shape)
print("Y_test shape = ", Y_test.shape)

X shape =  (60000, 28, 28, 1)
Y shape =  (60000,)
X_test shape =  (10000, 28, 28, 1)
Y_test shape =  (10000,)


In [18]:
classes_num = len(np.unique(train_labels))

In [14]:
class IdentityBlock(tf.keras.Model):
    def __init__(self, filters, kernel_size):
        super(IdentityBlock, self).__init__(name='')

        self.conv1 = tf.keras.layers.Conv2D(filters, kernel_size, padding='same')
        self.bn1 = tf.keras.layers.BatchNormalization()

        self.conv2 = tf.keras.layers.Conv2D(filters, kernel_size, padding='same')
        self.bn2 = tf.keras.layers.BatchNormalization()

        self.act = tf.keras.layers.Activation('relu')
        self.add = tf.keras.layers.Add()
    
    def call(self, input_tensor):
        x = self.conv1(input_tensor)
        x = self.bn1(x)
        x = self.act(x)

        x = self.conv2(x)
        x = self.bn2(x)
        x = self.act(x)

        x = self.add([x, input_tensor])
        x = self.act(x)
        return x

In [15]:
class ConvolutionBlock(tf.keras.Model):
    def __init__(self, filters, kernel_size):
        super(ConvolutionBlock, self).__init__(name='')

        self.conv1 = tf.keras.layers.Conv2D(filters, kernel_size, padding='same')
        self.bn1 = tf.keras.layers.BatchNormalization()

        self.conv2 = tf.keras.layers.Conv2D(filters, kernel_size, padding='valid')
        self.bn2 = tf.keras.layers.BatchNormalization()
        
        self.shortcut = tf.keras.layers.Conv2D(filters, kernel_size, padding='valid')
        self.shrtbn = tf.keras.layers.BatchNormalization()

        self.act = tf.keras.layers.Activation('relu')
        self.add = tf.keras.layers.Add()
    
    def call(self, input_tensor):
        x = self.conv1(input_tensor)
        x = self.bn1(x)
        x = self.act(x)

        x = self.conv2(x)
        x = self.bn2(x)
        x = self.act(x)
        
        shortcut = self.shortcut(input_tensor)
        shortcut = self.shrtbn(shortcut)

        x = self.add([x, shortcut])
        x = self.act(x)
        return x

In [16]:
class ResNet(tf.keras.Model):
    def __init__(self, num_classes):
        super(ResNet, self).__init__()
        self.conv = tf.keras.layers.Conv2D(64, 7, padding='same')
        self.bn = tf.keras.layers.BatchNormalization()
        self.act = tf.keras.layers.Activation('relu')
        self.max_pool = tf.keras.layers.MaxPool2D((3, 3))

        self.conv1a = ConvolutionBlock(64, 3)
        self.id1a = IdentityBlock(64, 3)
        self.conv1b = ConvolutionBlock(64, 3)
        self.id1b = IdentityBlock(64, 3)

        self.global_pool = tf.keras.layers.GlobalAveragePooling2D()
        self.classifier = tf.keras.layers.Dense(num_classes, activation='softmax')

    def call(self, inputs):
        x = self.conv(inputs)
        x = self.bn(x)
        x = self.act(x)
        x = self.max_pool(x)

        x = self.conv1a(x)
        x = self.id1a(x)
        x = self.conv1b(x)
        x = self.id1b(x)

        x = self.global_pool(x)
        return self.classifier(x)

In [19]:
resnet = ResNet(classes_num)
resnet.compile(optimizer='adam', loss='sparse_categorical_crossentropy', metrics=['accuracy'])
resnet.fit(x=X, y=Y, validation_data = (X_test, Y_test), epochs = 25, batch_size = 1024)

Epoch 1/25
59/59 [==============================] - 12s 107ms/step - loss: 0.5916 - accuracy: 0.7875 - val_loss: 2.0474 - val_accuracy: 0.3004
Epoch 2/25
59/59 [==============================] - 5s 87ms/step - loss: 0.3326 - accuracy: 0.8777 - val_loss: 0.6079 - val_accuracy: 0.7953
Epoch 3/25
59/59 [==============================] - 5s 87ms/step - loss: 0.2776 - accuracy: 0.8988 - val_loss: 0.3824 - val_accuracy: 0.8680
Epoch 4/25
59/59 [==============================] - 5s 87ms/step - loss: 0.2397 - accuracy: 0.9134 - val_loss: 0.3388 - val_accuracy: 0.8793
Epoch 5/25
59/59 [==============================] - 5s 87ms/step - loss: 0.2229 - accuracy: 0.9190 - val_loss: 0.3407 - val_accuracy: 0.8740
Epoch 6/25
59/59 [==============================] - 5s 87ms/step - loss: 0.2029 - accuracy: 0.9251 - val_loss: 0.3605 - val_accuracy: 0.8732
Epoch 7/25
59/59 [==============================] - 5s 87ms/step - loss: 0.1866 - accuracy: 0.9324 - val_loss: 0.2996 - val_accuracy: 0.8960
Epoch 8/25


In [20]:
resnet.summary()

Model: "res_net"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 conv2d (Conv2D)             multiple                  3200      
                                                                 
 batch_normalization (BatchN  multiple                 256       
 ormalization)                                                   
                                                                 
 activation (Activation)     multiple                  0         
                                                                 
 max_pooling2d (MaxPooling2D  multiple                 0         
 )                                                               
                                                                 
  (ConvolutionBlock)         multiple                  111552    
                                                                 
  (IdentityBlock)            multiple                  7436